<img src="../Imagenes/portada.jpg">

# Proyecto Web - NFL

## Descripción del proyecto

Hay muchas columnas que elimino del data set por resultar poco interesantes (y porque la RAM no me da).

Compararemos las estadisticas de juego con las de delitos cometidos por los jugadores de la NFL.

In [1]:
#Librerias
import numpy as np
import pandas as pd
import re
import http.client
import requests
import urllib,json
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, interact_manual

### Estrategia

In [53]:
#Funciones
def soups (url):
    """
    **************************************
    Llama a la url y le pasa BeautifulSoup
    **************************************
    """
    html = requests.get(url).content
    soup = BeautifulSoup(html, "lxml")
    return soup

def requestAPIs (url):
    """
    ***************************************
    Llama a la url de la API y normaliza el
    JSON
    ***************************************
    """
    headers = {'User-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'}
    request = requests.get(url, headers = headers)
    result = request.json()
    data = json_normalize(result)
    return data

def clean (words):
    
    """
    ****************************************************
    Función para limpiar la información,
    pasar la primera a mayúculas y el resto a minúscula,
    eliminaremos espacios innecesarios, el resto serán
    sustituidos por '_', igual que otros elementos de
    puntuación?
    
    Input : lista
    Output : lista
    ****************************************************
    
    """
    c = []
    for word in words:
        word = str(word).lower()
        word = word.strip()
        word = word.replace(".", ' ')
        word = word.replace(" ", '_')
        word = word.replace(":", '_')
        word = word.replace("__", '_')
        word = word.replace ("/", "")
        word = word.replace ("?", "")
        word = word.replace ("(", "")
        word = word.replace (")", "")
        
        c.append (word)
    return c

def change_names (list1, list2):
    
    """
    *****************************************
    Convierte a diccionario dos listas dadas
    *****************************************
    """
    
    dictchange = dict(zip(list(list1), list(list2)))
    return dictchange

def players_stats(url):
    
    """
    **************************************************
    Con esta función extraigo la tabla de estadísticas
    generales de la web de la NFL.
    ¡Está por optimizar!
    **************************************************
    """
    #Paso la función 'soups'
    soup = soups(url)
    table = soup.find_all('div',{'id': 'player-stats-wrapper'})[0]
    text = [element.text.strip().split("\n") for element in table.find_all('tr')]
    rows = [list(filter(None, row))for row in text]
    #Nombro las columnas para poder llamarlas luego y eliminar los None que aparecen
    df = pd.DataFrame(rows, columns = ['a0','a1', 'a2', 'a3', 'a4','a5', 'a6', 'a7', 'a8',
                                      'a9', 'a10', 'a11', 'a12','a13', 'a14', 'a15', 'a16',
                                      'a17', 'a18', 'a19'])

    #Seguro que todos estos drops se pueden hacer de forma más efectiva.
    df = df.drop(['a3', 'a5', 'a6', 'a7'], axis=1)
    df = df.drop_duplicates()
    #La 4ta fila me sale con Nones.
    df = df.drop(4, axis= 0)
    #Con esto elimino las filas superiores que no tienen info y la última
    df = df[3:-1]
    colnames = ['Season','Team','G','GS','Comb',
                'Total','Ast','Sck','SFTY','PDef',
                'Int','Yds','Avg', 'Lng', 'TDs','FF']
    #Paso las funciones 'clean' y 'change_names' para los cambios de nombre
    df = df.rename(columns = change_names(df.columns, clean(colnames)))

    df.gs = [re.sub("\t","", i) for i in df.gs]
    
    return df

## API NFLArrest

### Jugadores

API con información relacionada con los arrestos a los jugadores de la NFL. Para más información se puede consultar toda la documentación <a href="http://nflarrest.com/api/">aquí</a>.

Usaremos la función creada para hacer las consultas a la API.

In [10]:
url="http://nflarrest.com/api/v1/player"
NFL_players = requestAPIs(url)

Sacaremos un primer listado con todos los jugadores que han cometido algún delito. 
Este JSON nos da información sobre:
 * <CODE>'Name'</CODE>, nombre completo del jugador.
 * <CODE>'Team'</CODE>, siglas del equipo.
 * <CODE>'Team_name'</CODE>, nombre completo del equipo.
 * <CODE>'Team_city'</CODE>, ciudad del equipo.
 * <CODE>'Position'</CODE>, posisición,
 * <CODE>'arrest_count'</CODE>, arrestos acumulados.
 
Vemos que los nombres de los encabezados varían ligeramente. Usaremos la función del anterior proyecto para unificarlos.

In [11]:
#Renombramos columnas
NFL_players = NFL_players.rename(columns = change_names(NFL_players.columns, clean(NFL_players.columns)))

In [12]:
NFL_players

,name,team,team_name,team_city,position,arrest_count
0,Kenny Britt,TEN,Titans,Nashville,WR,7
1,Adam Jones,TEN,Titans,Nashville,CB,6
2,Chris Henry,CIN,Bengals,Cincinnati,WR,6
3,Aldon Smith,SF,49ers,San Francisco,LB,5
4,Fred Davis,WAS,Redskins,Washington DC,TE,4
...,...,...,...,...,...,...
106,Mike Nattiel,MIN,Vikings,Minneapolis,LB,2
107,William Moore,ATL,Falcons,Atlanta,S,2
108,Shaun Phillips,LAC,Chargers,Los Angeles,LB,2
109,Kevin Faulk,NE,Patriots,Boston,RB,2


Hay un total de 111 jugadores que han comitido algún delito.

In [16]:
@interact
# El @interact es necesario para que funcione.
def describe(column=list(NFL_players.columns)):
    
    """
    ********************************************
    Con esta función creamos un desplegable, al 
    seleccionar una de las columnas nos devolverá
    los datos agregados
    *********************************************
    """
    
    print(NFL_players[column].describe())

interactive(children=(Dropdown(description='column', options=('name', 'team', 'team_name', 'team_city', 'posit…

### Equipos

In [17]:
teams = list(pd.unique(NFL_players['team']))

In [18]:
lst = []

# Your code
for i in teams:
    #Asignamos a una variable un iterador que vaya aumentando hasta llegar al total de equipos de la API.
    #Luego la pasamos en la url para que vaya sacando la información de todas las páginas
    url = "http://nflarrest.com/api/v1/team/arrests/"
    url = (url+i)
    data = requestAPIs(url)
    lst.append(data)
    
NFL_teams = pd.concat(lst, sort= False, ignore_index = True)

#Renombramos las columnas
NFL_teams = NFL_teams.rename(columns = change_names(NFL_teams.columns, clean(NFL_teams.columns)))

In [19]:
NFL_teams.columns

Index(['arrest_stats_id', 'date', 'team', 'team_name', 'team_preffered_name',
       'team_city', 'team_logo_id', 'team_conference', 'team_division',
       'team_conference_division', 'team_hex_color', 'team_hex_alt_color',
       'reddit_group_id', 'name', 'position', 'position_name', 'position_type',
       'encounter', 'category', 'crime_category', 'crime_category_color',
       'description', 'outcome', 'season', 'arrestseasonstate',
       'overall_category_id', 'general_category_id', 'legal_level_id',
       'resolution_category_id', 'year', 'month', 'day', 'day_of_week',
       'day_of_week_int', 'yeartodate', 'dayssince', 'daystolastarrest',
       'daystolastcrimearrest', 'daystolastteamarrest'],
      dtype='object')

### Crimenes por jugador

Extraemos los nombres únicos de la primera tabla y los pasamos al formato adecuado para poderlos usar en la iteración.

In [20]:
names = pd.unique(NFL_teams['name'])
names = [i.replace(" ", "%20") for i in names]

In [21]:
lst = []
for i in names:
    # Asignamos a una variable un iterador que vaya aumentando hasta 
    # llegar al total de jugadores que han cometido delitos de la API.
    # Luego la pasamos en la url para que vaya sacando la información de todas las páginas.
    url = "http://nflarrest.com/api/v1/player/arrests/"
    url = (url+i)
    data = requestAPIs(url)
    lst.append(data)

In [22]:
NFLplayers_arrest = pd.concat(lst, sort= False, ignore_index = True)
NFLplayers_arrest = NFLplayers_arrest.rename(columns = change_names
                                             (NFLplayers_arrest.columns, clean(NFLplayers_arrest.columns)))

In [60]:
NFLplayers_arrest.head()

,arrest_stats_id,date,team,team_name,team_preffered_name,team_city,team_logo_id,team_conference,team_division,team_conference_division,...,year,month,day,day_of_week,day_of_week_int,yeartodate,dayssince,daystolastarrest,daystolastcrimearrest,daystolastteamarrest
0,803,2015-09-30,TEN,Titans,Tennessee Titans,Nashville,31,AFC,South,AFC South,...,2015,9,30,Wednesday,4,0,1603,35,341,72
1,3,2015-07-20,TEN,Titans,Tennessee Titans,Nashville,31,AFC,South,AFC South,...,2015,7,20,Monday,2,0,1675,0,167,269
2,33,2014-10-24,TEN,Titans,Tennessee Titans,Nashville,31,AFC,South,AFC South,...,2014,10,24,Friday,6,0,1944,2,126,217
3,68,2014-03-21,TEN,Titans,Tennessee Titans,Nashville,31,AFC,South,AFC South,...,2014,3,21,Friday,6,0,2161,12,91,391
4,132,2013-02-23,TEN,Titans,Tennessee Titans,Nashville,31,AFC,South,AFC South,...,2013,2,23,Saturday,7,0,2552,3,3,218


In [61]:
NFLplayers_arrest.shape

(870, 39)

In [62]:
NFL_teams.shape

(877, 39)

In [63]:
NFLarrest = pd.concat([NFLplayers_arrest, NFL_teams])

In [64]:
NFLarrest.columns

Index(['arrest_stats_id', 'date', 'team', 'team_name', 'team_preffered_name',
       'team_city', 'team_logo_id', 'team_conference', 'team_division',
       'team_conference_division', 'team_hex_color', 'team_hex_alt_color',
       'reddit_group_id', 'name', 'position', 'position_name', 'position_type',
       'encounter', 'category', 'crime_category', 'crime_category_color',
       'description', 'outcome', 'season', 'arrestseasonstate',
       'overall_category_id', 'general_category_id', 'legal_level_id',
       'resolution_category_id', 'year', 'month', 'day', 'day_of_week',
       'day_of_week_int', 'yeartodate', 'dayssince', 'daystolastarrest',
       'daystolastcrimearrest', 'daystolastteamarrest'],
      dtype='object')

In [65]:
#Borro muchas columnas que podrían ser interesantes, es que no me da la RAM
NFLarrest = NFLarrest.drop(['team_logo_id','team_conference_division', 
                            'team_hex_color', 'team_hex_alt_color',
                            'reddit_group_id','team_conference_division','encounter',
                            'crime_category_color','overall_category_id', 'general_category_id',
                            'legal_level_id', 'resolution_category_id','year','date',
                            'day_of_week_int', 'yeartodate', 'dayssince', 'daystolastarrest',
                            'daystolastcrimearrest', 'daystolastteamarrest'], axis = 1)

In [66]:
NFLarrest = NFLarrest.drop_duplicates()

In [67]:
NFLarrest.shape

(885, 20)

Con esto podemos ver los jugadores con más delitos cometidos

In [82]:
NFLarrest['name'].value_counts()

Adam Jones           10
Kenny Britt           7
Chris Henry           6
Aldon Smith           6
Bryant McKinnie       4
                     ..
Cortland Finnegan     1
Roddy White           1
Matt Schaub           1
Ben Kelly             1
Charlie Rogers        1
Name: name, Length: 665, dtype: int64

Observamos que sale un número diferente a la primera tabla extraída. Lo comprobaremos con el <CODE>'arrest_stats_id'</CODE>

In [32]:
NFLarrest[NFLarrest.name == 'Adam Jones']

,arrest_stats_id,team,team_name,team_preffered_name,team_city,team_conference,team_division,name,position,position_name,position_type,category,crime_category,description,outcome,season,arrestseasonstate,month,day,day_of_week
19,830,CIN,Bengals,Cincinnati Bengals,Cincinnati,AFC,North,Adam Jones,CB,Cornerback,D,Assault,Assault / Battery,Accused of poking hotel security employee in t...,"Pleaded guilty to misdemeanor charge, sentence...",2017,OffSeason,1,2,Monday
20,92,CIN,Bengals,Cincinnati Bengals,Cincinnati,AFC,North,Adam Jones,CB,Cornerback,D,Disorderly conduct,Alcohol / Disorderly,He was the passenger in a car pulled over for ...,Paid $130 for fine and court costs.,2013,InSeason,9,23,Monday
21,107,CIN,Bengals,Cincinnati Bengals,Cincinnati,AFC,North,Adam Jones,CB,Cornerback,D,Assault,Assault / Battery,Accused of punching a woman outside a Cincinna...,Resolution undetermined.,2013,OffSeason,6,10,Monday
22,207,CIN,Bengals,Cincinnati Bengals,Cincinnati,AFC,North,Adam Jones,CB,Cornerback,D,Alcohol,Alcohol / Disorderly,Accused of disorderly conduct while intoxicate...,Resolution undetermined.,2011,OffSeason,7,10,Sunday
23,440,TEN,Titans,Tennessee Titans,Nashville,AFC,South,Adam Jones,CB,Cornerback,D,"Coercion, gun",Other,"Surrendered in Las Vegas, charged with felony ...",Pleaded no contest to conspiracy to commit dis...,2007,OffSeason,6,21,Thursday
24,487,TEN,Titans,Tennessee Titans,Nashville,AFC,South,Adam Jones,CB,Cornerback,D,Assault,Assault / Battery,Accused of spitting in a woman's face at Nashv...,Charge dropped because of inconsistent testimony.,2006,InSeason,10,26,Thursday
25,500,TEN,Titans,Tennessee Titans,Nashville,AFC,South,Adam Jones,CB,Cornerback,D,Public intoxication,Alcohol / Disorderly,"Accused of being drunk, shouting profanities a...",Charges dropped on the condition he stays out ...,2006,OffSeason,8,25,Friday
26,534,TEN,Titans,Tennessee Titans,Nashville,AFC,South,Adam Jones,CB,Cornerback,D,Drugs,Drugs,Accused of smoking marijuana after he emerged ...,Charge dropped.,2006,OffSeason,3,23,Thursday
27,541,TEN,Titans,Tennessee Titans,Nashville,AFC,South,Adam Jones,CB,Cornerback,D,Drugs,Drugs,Accused of throwing a punch at an officer and ...,"Pleaded no contest to obstructing police, thre...",2006,OffSeason,2,6,Monday
28,568,TEN,Titans,Tennessee Titans,Nashville,AFC,South,Adam Jones,CB,Cornerback,D,Assault,Assault / Battery,Accused of assault and vandalism after being a...,Charges dropped after nightclub manager said h...,2005,OffSeason,7,13,Wednesday


## Web NFL

Seleccionamos a los 3 jugadores con un mayor número de arrestos para ver sus estadísiticas de juego.

### Adam Jones

In [51]:
url = "http://www.nfl.com/player/adamjones/2506345/profile"

In [54]:
AdamJones = players_stats(url)

In [55]:
AdamJones

,season,team,g,gs,comb,total,ast,sck,sfty,pdef,int,yds,avg,lng,tds,ff
3,2018,Denver Broncos,7,2,9,9,0,0.0,--,3,1,0,0.0,0,0,0
5,2017,Cincinnati Bengals,9,9,23,19,4,0.0,--,4,1,0,0.0,0,0,0
7,2016,Cincinnati Bengals,16,16,66,54,12,0.0,--,7,1,2,2.0,2,0,1
9,2015,Cincinnati Bengals,14,14,62,53,9,1.0,--,12,3,14,4.7,14,0,1
11,2014,Cincinnati Bengals,16,3,63,51,12,0.0,--,11,3,23,7.7,12,0,0
13,2013,Cincinnati Bengals,16,13,56,44,12,0.0,0,12,3,60,20.0,60T,1,1
15,2012,Cincinnati Bengals,16,5,40,33,7,1.0,0,9,--,--,0.0,--,--,1
17,2011,Cincinnati Bengals,8,7,28,26,2,0.0,--,6,--,--,0.0,--,--,0
19,2010,Cincinnati Bengals,5,1,14,11,3,0.0,0,3,1,10,10.0,10,0,1
21,2008,Dallas Cowboys,9,6,31,26,5,0.0,--,7,--,--,0.0,--,--,1


### Kenny Britt

In [56]:
url = 'http://www.nfl.com/player/kennybritt/71217/profile'

In [57]:
KennyBritt = players_stats(url)

In [58]:
KennyBritt

,season,team,g,gs,comb,total,ast,sck,sfty,pdef,int,yds,avg,lng,tds,ff
3,2017,New England Patriots,3,0,2,23,11.5,16,0,--,--,--,--,--,--,--
5,2017,Cleveland Browns,9,4,18,233,12.9,38,2,--,--,--,--,--,--,--
7,2016,Los Angeles Rams,15,15,68,"1,002",14.7,66,5,--,--,--,--,--,2,1
9,2015,St. Louis Rams,16,14,36,681,18.9,60T,3,--,--,--,--,--,--,--
11,2014,St. Louis Rams,16,13,48,748,15.6,63T,3,2,14,7.0,12,0,--,--
13,2013,Tennessee Titans,12,3,11,96,8.7,15,0,--,--,--,--,--,--,--
15,2012,Tennessee Titans,14,11,45,589,13.1,46,4,--,--,--,--,--,2,1
17,2011,Tennessee Titans,3,3,17,289,17.0,80T,3,--,--,--,--,--,1,1
19,2010,Tennessee Titans,12,7,42,775,18.5,80T,9,--,--,--,--,--,1,1
21,2009,Tennessee Titans,16,6,42,701,16.7,57,3,--,--,--,--,--,1,1


### Chris Henry

In [43]:
url = 'http://www.nfl.com/player/chrishenry/2506422/profile'

In [44]:
ChrisHenry = players_stats(url)

In [59]:
ChrisHenry

,Season,Team,G,GS,Comb,Total,Ast,Sck,SFTY,PDef,Int,Yds,Avg,Lng,TDs,FF,Name
3,2009,Cincinnati Bengals,8,0,12,236,19.7,73,2,--,--,--,--,--,--,--,Chris Henry
5,2008,Cincinnati Bengals,12,1,19,220,11.6,22,2,--,--,--,--,--,--,--,Chris Henry
7,2007,Cincinnati Bengals,8,2,21,343,16.3,52T,2,--,--,--,--,--,--,--,Chris Henry
9,2006,Cincinnati Bengals,13,4,36,605,16.8,71,9,--,--,--,--,--,--,--,Chris Henry
11,2005,Cincinnati Bengals,14,5,31,422,13.6,47,6,--,--,--,--,--,2,1,Chris Henry


## Resultados

Exportamos los dos .csv

### CSV API

In [ ]:
NFLarrest.to_csv("../Datos/NFLarrest.csv", index = False)

### CSV Web

Para poder unir los dataframes de los diferentes jugadores hemos añadido otra columna con los nombres.

In [46]:
AdamJones['Name'] = "Adam Jones"
KennyBritt['Name'] = "Kenny Britt"
ChrisHenry['Name'] = "Chris Henry"

In [47]:
BadPlayersStats = pd.concat([AdamJones, KennyBritt, ChrisHenry])

In [49]:
BadPlayersStats.to_csv("../Datos/BadPlayersStats.csv", index = False)